Part 1
Encryption and decryption using built-in AES in openSSL library

In [1]:
import os
from Crypto.Cipher import AES

def encrypt_file(key, in_filename, out_filename=None, chunksize=16):
    if not out_filename:
        out_filename = in_filename + '.enc'
    
    encryptor = AES.new(key, AES.MODE_ECB)
    filesize = os.path.getsize(in_filename)

    with open(in_filename, 'rb') as infile:
        with open(out_filename, 'wb') as outfile:
            while True:
                chunk = infile.read(chunksize)
                if len(chunk) == 0:
                    break
                elif len(chunk) % 16 != 0:
                    chunk +=b' ' * (16 - len(chunk) %16)

                outfile.write(encryptor.encrypt(chunk))

def decrypt_file(key, in_filename, out_filename=None, chunksize=16):
    if not out_filename:
        out_filename = os.path.splitext(in_filename)[0]
    
    with open(in_filename, 'rb') as infile:
        decryptor = AES.new(key, AES.MODE_ECB)

        with open(out_filename, 'wb') as outfile:
            while True:
                chunk = infile.read(chunksize)
                if len(chunk) == 0:
                    break
                outfile.write(decryptor.decrypt(chunk))

kk = b"1234567812345678"
encrypt_file(kk, 'in.txt', 'out.txt')
decrypt_file(kk, 'out.txt', 'dec.txt')

Part 2
- Use RSA for signing and  validation (which is authentication)  

In [1]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.asymmetric import padding

def generate_key_pair():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()
    return private_key, public_key

def sign_message(private_key, message):
    signature = private_key.sign(
        message,
        padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256()
    )
    return signature

def verify_signature(public_key, message, signature):
    try:
        public_key.verify(
            signature,
            message,
            padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            hashes.SHA256()
        )
        print("Signature verification successful. Message is authentic.")
    except Exception as e:
        print("Signature verification failed. Message may be tampered:", str(e))

def read_from_file(file_path):
    with open(file_path, 'rb') as file:
        return file.read()
    
def write_to_file(file_path, data):
    with open(file_path, 'wb') as file:
        file.write(data)
    

if __name__ == "__main__":
    # Read plaintext from input file
    input_text = read_from_file('in.txt')

    # Generate key pair
    private_key, public_key = generate_key_pair()

    # Sign the message
    signature = sign_message(private_key, input_text)
    write_to_file('signature.txt', signature)

    signature_in = read_from_file('signature.txt')
    # Check Signature verification successful
    verify_signature(public_key, input_text, signature_in)


Signature verification successful. Message is authentic.


Part 3
======================
- Calculate the hash of plaintext
- Encrypt hash with private key
- Append result to plaintext
- Encrypt everything with symmetric key
This is signing + encryption

In [3]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

def generate_keypair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

def sign_data(private_key, data):
    signature = private_key.sign(
        data,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    return signature

def encrypt_data(sym_key, data):
    iv = os.urandom(16)  # Initialization Vector for AES
    cipher = Cipher(algorithms.AES(sym_key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(data) + encryptor.finalize()
    return iv + ciphertext

def decrypt_data(sym_key, encrypted_data):
    iv = encrypted_data[:16]
    ciphertext = encrypted_data[16:]
    cipher = Cipher(algorithms.AES(sym_key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    return decryptor.update(ciphertext) + decryptor.finalize()

def verify_signature(public_key, message_hash, signature):
    decrypted_data_hash = hash_data(message_hash)
    try:
        public_key.verify(
            signature,
            decrypted_data_hash,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        print("Signature verification successful. Message is authentic.")
    except Exception as e:
        print("Signature verification failed. Message may be tampered:", str(e))

def hash_data(data):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(data)
    return digest.finalize()

def read_from_file(file_path):
    with open(file_path, 'rb') as file:
        return file.read()

def write_to_file(file_path, data):
    with open(file_path, 'wb') as file:
        file.write(data)

def main():
    # Read plaintext from input file
    input_text = read_from_file('in.txt')

    # Generate keys
    private_key, public_key = generate_keypair()
    
    # Hash the plaintext for signing
    plaintext_hash = hash_data(input_text)
    signature = sign_data(private_key, plaintext_hash)

    # Combine encrypted data and signature
    combined_data = input_text + signature

    # Encrypt the combined message
    symmetric_key_outer = os.urandom(32)  # 256-bit key for AES
    encrypted_combined_data = encrypt_data(symmetric_key_outer, combined_data)

    write_to_file('Encrypted_Combined_Data.txt', encrypted_combined_data)
    input_com = read_from_file('Encrypted_Combined_Data.txt')

    # Decrypt the combined message
    decrypted_combined_data = decrypt_data(symmetric_key_outer, input_com)

    # Extract the original encrypted data and signature
    original_encrypted_data = decrypted_combined_data[:-256]
    extracted_signature = decrypted_combined_data[-256:]

    # Verify the signature against the hash of the decrypted data
    verify_signature(public_key, original_encrypted_data, extracted_signature)

    # Write the decrypted data to the output file
    write_to_file('output.txt', original_encrypted_data)
    

    print("Original Data:", input_text.decode('utf-8'))
    print("Encrypted Combined Data:", encrypted_combined_data.hex())
    print("Decrypted Data:", original_encrypted_data.decode('utf-8'))

if __name__ == "__main__":
    main()


Signature verification successful. Message is authentic.
Original Data: This is a sample plaintext for testing encryption and decryption.
Encrypted Combined Data: 13c85e6148f7abc06d27bce08abe7c4cfa9e7f25b931bc5aacd9c7cc8f800981144de9497b59d56ab1ce42abfb2febe4a8306b6e5dae83fb6c7cf9c261f401f4cc2482a2d10e9bf3ac160401451b4dc9af8dfce4c73dfee4a372e0585ec363474dd1f4fc972d5e10cc943a10f0e9313cc201ec71daeabb5dd010b0b0c1e60f9b7523c9d6576d6531a5d6a920fe94e19a5a52d32b4ea75e65bd7a4b467c418f7d6b5a24eb5cafbaf2fc42d232850ce05c1587fbed0f9ccd0c08bded0ed30cc89c7d5ff2df51e1a7339557900e81c4b5bdef62a15e1db1638d2536f3b6ab10de352a9b838d767102735a780a7cab18d90682e267a18b9b0b1af07cc251531c68a22b43696625d873be693c95bfcc89ea6e159e48beb1a1b220ac63118f7e6a99387c9e92ecb0c07992f55bd25586adbc6b31197e0037e64c388dc2bdcaece26f7cb566b4e352fa0a1224ddb735a62f4d818d
Decrypted Data: This is a sample plaintext for testing encryption and decryption.
